In [1]:
SEASON = '2019-20'

In [2]:
import pandas as pd
import myconstants as c
from scipy.stats import zscore

NUM_GAMES = c.NUM_GAMES[SEASON]
MIN_GAMES = NUM_GAMES * 0.60

p_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_PerGame.csv')
t_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Team_Standings.csv')

# join player per game and advanced stats
p_advanced_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_Player_Advanced.csv')
p_advanced_df = p_advanced_df.drop(columns=['Player', 'Tms', 'Pos', 'Age', 'G', 'MP'])
p_df = p_df.merge(p_advanced_df, on='PlayerID')

# populate team df with team codes
t_df['TeamCode'] = t_df['Team'].apply(lambda x: c.TEAM_CODE[x])

# average team wins per player
def average_wins(tms):
    tms_list = tms.split(',')
    tms_wins = []
    for t in tms_list:
        tms_wins.append(t_df.loc[t_df['TeamCode'] == t]['OverallW'].values[0])
    return sum(tms_wins) / len(tms_wins)
p_df['TmsWins'] = p_df['Tms'].apply(average_wins)

# players who haven't played enough games don't qualify
p_df = p_df.loc[p_df['G'] >= MIN_GAMES]

# ranking algorithm
p_df['PlayerScore'] = 1.5*zscore(p_df['PTS'] + p_df['TRB'] + p_df['AST'] + p_df['STL'] + p_df['BLK']) + zscore(p_df['WS']) + zscore(p_df['VORP']) + zscore(p_df['BPM']) + zscore(p_df['TmsWins'])
p_df = p_df.sort_values('PlayerScore', ascending=False)

# Move [PlayerScore] to the right of [Pos], drop PlayerID, index rows starting from 1, stylistic choice
col = p_df.pop('PlayerScore')
p_df.insert(p_df.columns.get_loc('Pos') + 1, col.name, col)
p_df = p_df.drop('PlayerID', axis=1)
p_df = p_df.reset_index(drop=True)
p_df.index = p_df.index + 1

p_df.head(50)

,Player,Tms,Pos,PlayerScore,Age,G,GS,MP,FG,FGA,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,TmsWins
1,Giannis Antetokounmpo,MIL,PF,19.308722,25,63,63,30.4,10.9,19.7,...,37.5,6.1,5.0,11.1,0.279,7.4,4.1,11.5,6.6,56.0
2,James Harden,HOU,SG,19.023285,30,68,68,36.5,9.9,22.3,...,36.3,9.9,3.2,13.1,0.254,8.1,1.6,9.6,7.3,44.0
3,LeBron James,LAL,PG,15.916669,35,67,67,34.6,9.6,19.4,...,31.5,6.2,3.6,9.8,0.204,6.6,1.8,8.4,6.1,52.0
4,Anthony Davis,LAL,PF,15.345914,26,62,62,34.4,8.9,17.7,...,29.3,6.7,4.4,11.1,0.250,5.4,2.6,8.0,5.4,52.0
5,Luka Dončić,DAL,PG,14.642132,20,61,61,33.6,9.5,20.6,...,36.8,6.5,2.3,8.8,0.207,7.4,1.0,8.4,5.4,43.0
6,Damian Lillard,POR,PG,14.474196,29,66,66,37.5,9.5,20.4,...,30.3,10.9,0.7,11.6,0.225,8.3,-0.9,7.5,5.9,35.0
7,Kawhi Leonard,LAC,SF,13.992148,28,57,57,32.4,9.3,19.9,...,33.0,5.4,3.3,8.7,0.226,6.5,2.4,8.9,5.1,49.0
8,Nikola Jokić,DEN,C,13.460522,24,73,73,32.0,7.7,14.7,...,26.6,6.7,3.1,9.8,0.202,5.5,2.0,7.4,5.5,46.0
9,Jimmy Butler,MIA,SF,10.201097,30,58,58,33.8,5.9,13.1,...,25.1,6.3,2.7,9.0,0.221,4.0,1.5,5.4,3.7,44.0
10,Rudy Gobert,UTA,C,9.604542,27,68,68,34.3,5.7,8.2,...,16.2,6.5,4.2,10.7,0.221,1.7,1.9,3.6,3.3,44.0
